In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

# Load and filter (same as original)
X_flux = np.load("X_flux_512.npy")[..., np.newaxis]  # (samples, 512, 1)
X_tabular = np.load("X_tabular.npy")
y = np.load("y.npy")
mask = y != -1
X_flux, X_tabular, y = X_flux[mask], X_tabular[mask], y[mask]

# Train/Val split
Xf_train, Xf_val, Xt_train, Xt_val, y_train, y_val = train_test_split(
    X_flux, X_tabular, y, test_size=0.2, stratify=y, random_state=42
)

# Normalize tabular data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xt_train = scaler.fit_transform(Xt_train)
Xt_val = scaler.transform(Xt_val)

# Apply SMOTE to tabular data
smote = SMOTE(random_state=42)
Xt_train_smote, y_train_smote = smote.fit_resample(Xt_train, y_train)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Focal Loss implementation
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

# Compute class weights for neural networks
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}



In [3]:

# 1. Logistic Regression (Tabular Only) with SMOTE
clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(Xt_train_smote, y_train_smote)
log_preds = clf.predict(Xt_val)

print("\n🔸 Logistic Regression (Tabular Only) with SMOTE")
log_report = classification_report(y_val, log_preds, target_names=["False Positive", "Confirmed"], output_dict=True)
print(classification_report(y_val, log_preds, target_names=["False Positive", "Confirmed"]))



🔸 Logistic Regression (Tabular Only) with SMOTE
                precision    recall  f1-score   support

False Positive       0.95      0.65      0.78       127
     Confirmed       0.44      0.90      0.59        39

      accuracy                           0.71       166
     macro avg       0.70      0.78      0.68       166
  weighted avg       0.83      0.71      0.73       166



In [4]:

# 2. MLP (Tabular Only) with Class Weighting and Focal Loss
class TabularMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(5, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

tabular_model = TabularMLP().to(device)
criterion = FocalLoss(alpha=0.75, gamma=2.0)
optimizer = torch.optim.Adam(tabular_model.parameters(), lr=1e-3)

Xtt_train = torch.tensor(Xt_train_smote, dtype=torch.float32).to(device)
ytt_train = torch.tensor(y_train_smote, dtype=torch.float32).unsqueeze(1).to(device)

# Training loop with early stopping
best_loss = float('inf')
patience = 10
counter = 0
for epoch in range(100):
    tabular_model.train()
    optimizer.zero_grad()
    output = tabular_model(Xtt_train)
    loss = criterion(output, ytt_train)
    loss.backward()
    optimizer.step()

    # Validation loss for early stopping
    tabular_model.eval()
    with torch.no_grad():
        val_output = tabular_model(torch.tensor(Xt_val, dtype=torch.float32).to(device))
        val_loss = criterion(val_output, torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device))
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter += 1
    if counter >= patience:
        break

tabular_model.eval()
with torch.no_grad():
    val_logits = tabular_model(torch.tensor(Xt_val, dtype=torch.float32).to(device))
    val_probs = torch.sigmoid(val_logits).cpu().numpy()
    val_preds = (val_probs > 0.5).astype(int)

print("\n🔸 MLP (Tabular Only) with Focal Loss")
mlp_report = classification_report(y_val, val_preds, target_names=["False Positive", "Confirmed"], output_dict=True)
print(classification_report(y_val, val_preds, target_names=["False Positive", "Confirmed"]))




🔸 MLP (Tabular Only) with Focal Loss
                precision    recall  f1-score   support

False Positive       0.99      0.85      0.92       127
     Confirmed       0.67      0.97      0.79        39

      accuracy                           0.88       166
     macro avg       0.83      0.91      0.85       166
  weighted avg       0.91      0.88      0.89       166



In [5]:

# 3. 1D CNN (Flux Only) with Class Weighting and Focal Loss
class FluxCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.cnn(x)

flux_model = FluxCNN().to(device)
criterion = FocalLoss(alpha=0.75, gamma=2.0)
optimizer = torch.optim.Adam(flux_model.parameters(), lr=1e-3)

Xff_train = torch.tensor(Xf_train, dtype=torch.float32).permute(0, 2, 1).to(device)
yff_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)

# Training loop with early stopping
best_loss = float('inf')
counter = 0
for epoch in range(100):
    flux_model.train()
    optimizer.zero_grad()
    output = flux_model(Xff_train)
    loss = criterion(output, yff_train)
    loss.backward()
    optimizer.step()

    # Validation loss for early stopping
    flux_model.eval()
    with torch.no_grad():
        val_output = flux_model(torch.tensor(Xf_val, dtype=torch.float32).permute(0, 2, 1).to(device))
        val_loss = criterion(val_output, torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device))
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter += 1
    if counter >= patience:
        break

flux_model.eval()
with torch.no_grad():
    val_flux_tensor = torch.tensor(Xf_val, dtype=torch.float32).permute(0, 2, 1).to(device)
    val_logits = flux_model(val_flux_tensor)
    val_probs = torch.sigmoid(val_logits).cpu().numpy()
    val_preds = (val_probs > 0.5).astype(int)

print("\n🔸 1D CNN (Flux Only) with Focal Loss")
cnn_report = classification_report(y_val, val_preds, target_names=["False Positive", "Confirmed"], output_dict=True)
print(classification_report(y_val, val_preds, target_names=["False Positive", "Confirmed"]))




🔸 1D CNN (Flux Only) with Focal Loss
                precision    recall  f1-score   support

False Positive       0.77      1.00      0.87       127
     Confirmed       0.00      0.00      0.00        39

      accuracy                           0.77       166
     macro avg       0.38      0.50      0.43       166
  weighted avg       0.59      0.77      0.66       166



c:\Users\Roshen Hasangha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Roshen Hasangha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Roshen Hasangha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

In [6]:

# 4. Enhanced Early Fusion (CNN + MLP) with Class Weighting and Focal Loss
class EnhancedEarlyFusion(nn.Module):
    def __init__(self):
        super().__init__()
        self.flux_cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten()
        )
        self.mlp = nn.Sequential(
            nn.Linear(64 + 5, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, flux, tabular):
        flux_features = self.flux_cnn(flux)
        combined = torch.cat((flux_features, tabular), dim=1)
        return self.mlp(combined)

fusion_model = EnhancedEarlyFusion().to(device)
criterion = FocalLoss(alpha=0.75, gamma=2.0)
optimizer = torch.optim.Adam(fusion_model.parameters(), lr=1e-3)

Xff_train = torch.tensor(Xf_train, dtype=torch.float32).permute(0, 2, 1).to(device)
Xtt_train = torch.tensor(Xt_train, dtype=torch.float32).to(device)
yff_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)

# Training loop with early stopping
best_loss = float('inf')
counter = 0
for epoch in range(100):
    fusion_model.train()
    optimizer.zero_grad()
    output = fusion_model(Xff_train, Xtt_train)
    loss = criterion(output, yff_train)
    loss.backward()
    optimizer.step()

    # Validation loss for early stopping
    fusion_model.eval()
    with torch.no_grad():
        val_output = fusion_model(
            torch.tensor(Xf_val, dtype=torch.float32).permute(0, 2, 1).to(device),
            torch.tensor(Xt_val, dtype=torch.float32).to(device)
        )
        val_loss = criterion(val_output, torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device))
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter += 1
    if counter >= patience:
        break

fusion_model.eval()
with torch.no_grad():
    val_logits = fusion_model(
        torch.tensor(Xf_val, dtype=torch.float32).permute(0, 2, 1).to(device),
        torch.tensor(Xt_val, dtype=torch.float32).to(device)
    )
    val_probs = torch.sigmoid(val_logits).cpu().numpy()
    val_preds = (val_probs > 0.5).astype(int)

print("\n🔸 Enhanced Early Fusion (CNN + MLP) with Focal Loss")
fusion_report = classification_report(y_val, val_preds, target_names=["False Positive", "Confirmed"], output_dict=True)
print(classification_report(y_val, val_preds, target_names=["False Positive", "Confirmed"]))


🔸 Enhanced Early Fusion (CNN + MLP) with Focal Loss
                precision    recall  f1-score   support

False Positive       0.77      1.00      0.87       127
     Confirmed       0.00      0.00      0.00        39

      accuracy                           0.77       166
     macro avg       0.38      0.50      0.43       166
  weighted avg       0.59      0.77      0.66       166



c:\Users\Roshen Hasangha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Roshen Hasangha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Roshen Hasangha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 